In [ ]:
import sympy as sm
import sympy.physics.mechanics as me
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve, minimize

from matplotlib import animation
import matplotlib
from matplotlib import patches
import matplotlib.pyplot as plt
from IPython.display import HTML
matplotlib.rcParams['animation.embed_limit'] = 2**128
%matplotlib inline

import time

Needed to exit the loop in the function *event* when a second contact point was found.

In [ ]:
class Rausspringen(Exception):
    pass

A homogeneous disc with radius r and mass m is running on an uneven 'street' without sliding. 
The disc is not allowed to jump, hence I can calculate the reaction forces needed to hold it on the
street at all times

The 'overall' shape of the street is modelled as a parabola (called strassen_form), the unevenness is 
modelled as a sum of sin functions (called strasse) with each term having a smaller amplitude and higher 
frequency as the previous one.
the 'street itself' is the sum of strasse and strassen_form, called gesamt.

When $r < | \dfrac{(1 + \left(\frac{d}{dx}(gesamt(x(t)))\right)^{3/2}} {\frac{d^2}{dx^2}\left(gesamt(x(t))\right)}|$, the disc will always have only one contact point. If this inequality does not hold, there may be a second contact point. (The formula for osculating radius (Krümmungsradius) is from Wikipedia.)\
The disc is supposed to run over these 'pot holes'. I use the key word **events** in solve_ivp to get the event: a second contact point is there.


**Variables**

- $q_1, u_1$: rotation of the disc and its speed
- $x, u_x$: loction of the contact point and the speed of successive contact points

- N: inertial frame
- $P_0$ point fixed in N
- $A_2$ body fixed frame of disc
- $Dmc$: center of disc
- $Dmc_o$: location of observer

- $m$: mass of the disc
- $m_o$: mass of the observer
- $i_{ZZ}$: moment of inertia of the disc aroud the Z axis
- $\alpha, \beta$: location of the observer relative to the center of the disc
- $amplitude, frequenz$: parameters of the street
- $reibung$: friction
- $rhs_1$: just a place holder for $MM^{-1}\cdot force[1]$ to be calculated later numerically. Needed for the rection force at CP


In [ ]:
start = time.time()

# q1 is the angle of the disc, x is the horizontal position of contact point CP
q1, x = me.dynamicsymbols('q1 x')  
u1, ux = me.dynamicsymbols('u1 ux')

auxx, auxy, fx, fy, rhs1 = me.dynamicsymbols('auxx auxy fx fy rhs1')   # for the reaction forces at contact point CP

m, mo, g, r, iZZ, alpha, beta = sm.symbols('m, mo, g, r, iZZ, alpha, beta')
amplitude, frequenz, reibung, t = sm.symbols('amplitude frequenz reibung t') # reibung = friction in German

N = me.ReferenceFrame('N')         # fixed inertial frame
A2 = me.ReferenceFrame('A2')       # fixed to the disc
P0, CP, Dmc, Dmco = sm.symbols('P0, CP, Dmc, Dmco', cls=me.Point)

Determine the street and its osculating radius (Schmiegekreis). As the disc should contact the street at exactly one point, this osculating radius must be larger than the radius of the disc. I found the formula for the osculating radius of a function in the internet.

In [ ]:
#============================================
rumpel = 4    # the higher the number the more 'uneven the street'
#============================================
def gesamt1(x, amplitude, frequenz):
    strasse = sum([amplitude/j * sm.sin(j*frequenz * x) for j in range(1, rumpel)])
    strassen_form = (frequenz/4. * x)**2
    return strassen_form + strasse
gesamt = gesamt1(x, amplitude, frequenz)

r_max = (sm.S(1.) + (gesamt.diff(x))**2 )**sm.S(3/2)/gesamt.diff(x, 2)

**Relationship of x(t) to q(t)**:

Obviously, $ x(t) = function(q(t), gesamt(x(t), r) $.
When the disc is rotated through an angle $q$, the arc length is $r\cdot q(t)$.

The arc length of a function f(k(t)) from 0 to $x(t)$ is: $ \int_{0}^{x(t)} \sqrt{1 +  \left(\frac{d}{dx}(f(k(t)\right)^2} \,dk \ $ (I found this in the internet)

This gives the sought after relationship between $q(t)$ and $x(t)$:
$ r \cdot (-q(t))  =  \int_{0}^{x(t)} \sqrt{1 + \left( \frac{d}{dk}(gesamt(k(t) \right)^2}\,dk \ $, differentiated w.r.t *t*:
- $ r \cdot (-u)  = \sqrt{1 + \left( \frac{d}{dx}(gesamt(x(t))\right)^2} \cdot \frac{d}{dt}(x(t) $, that is solved for $\frac{d}{dt}\left(x(t)\right)$:

- $\frac{d}{dt}(x(t)) = \dfrac{-(r \cdot u)} {\sqrt{1 + \left(\frac{d}{dx}(gesamt(x(t)\right)^2}}\$

The - sign is a consequence of the 'right hand rule' for frames. This is the sought after first order differential equation for $x(t)$.

In [ ]:
rhs3 = (-u1 * r / sm.sqrt(1. + (gesamt1(x, amplitude, frequenz).diff(x))**2)).simplify()

The vector perpendicular to the strasse is -($\frac{d}{dx}gesamt(x), - 1$). I found this in the internet.\
The leading minus sign, because directed 'inward'. It points from the contact point CP to the geometric center of the disc Dmc. 

In [ ]:
#The center of the wheel is at distance r from CP, perpendicular to the surface of the street.
vector = (-(gesamt1(x, amplitude, frequenz).diff(x)*N.x - N.y)).simplify()

A2.orient_axis(N, q1, N.z)
A2.set_ang_vel(N, u1 * N.z)

CP.set_pos(P0, x*N.x + gesamt1(x, amplitude, frequenz)*N.y)    # location of contact point
Dmc.set_pos(CP, r * (vector.normalize()).simplify())
TEST = Dmc.pos_from(P0)
print('Dmc DS', me.find_dynamicsymbols(TEST, reference_frame=N))
print('Dmc FS', TEST.free_symbols(reference_frame=N))

CP.set_vel(N, (CP.pos_from(P0).diff(t, N)).subs({sm.Derivative(x, t): rhs3}))
Dmc.set_vel(N, Dmc.pos_from(P0).diff(t, N).subs({sm.Derivative(x, t): rhs3}) + auxx*N.x + auxy*N.y)
print('Dmc DS', me.find_dynamicsymbols(Dmc.vel(N), reference_frame=N))
print('Dmc FS', Dmc.vel(N).free_symbols(reference_frame=N))


Dmco.set_pos(Dmc, r * (alpha*A2.x + beta*A2.y))
Dmco.set_vel(N, Dmco.pos_from(P0).diff(t, N).subs({sm.Derivative(x, t): rhs3, sm.Derivative(q1, t): u1}))

# just needed for potting later
Dmco_pos = [me.dot(Dmco.pos_from(P0), uv) for uv in (N.x, N.y)]
Dmc_pos = [me.dot(Dmc.pos_from(P0), uv) for uv in (N.x, N.y)]
CP_pos = [me.dot(CP.pos_from(P0), uv) for uv in (N.x, N.y)] # just for the plotting

I need this simple function to later see, if there is a **second contact point**, $CP_2$, at position ($xh, gesamt(xh, frequenz, amplitude, rumpel)$\
I only need to look into the direction the disc is moving, and only the interval $(x, 2\cdot r]$ 

In [ ]:
xh  = sm.symbols('xh')
CP2 = me.Point('CP2')
CP2.set_pos(P0, xh*N.x + gesamt1(xh, amplitude, frequenz)*N.y)
abstand2 = Dmc.pos_from(CP2).magnitude()
print('abstand2 DS', me.find_dynamicsymbols(abstand2))
print('abstand2 FS', abstand2.free_symbols)

abstand2_lam = sm.lambdify([x, xh, r, amplitude, frequenz], abstand2, cse=True)

Define the bodies and the energies.
I find it useful to calculate and plot the energies: They may give hints, that there may be a mistake in setting up Kane's equations - like it happened with almost every example I try to set up.

In [ ]:
I = me.inertia(A2, 0., 0., iZZ)                                              
Body = me.RigidBody('Body', Dmc, A2, m, (I, Dmc))
observer = me.Particle('observer', Dmco, mo)
BODY = [Body, observer]
kin_energie = (Body.kinetic_energy(N) + observer.kinetic_energy(N)).subs({auxx: 0., auxy:  0.})
pot_energie = m * g * me.dot(Dmc.pos_from(P0), N.y) + mo * g * me.dot(Dmco.pos_from(P0), N.y) 

print('kinetic energy DS: ', me.find_dynamicsymbols(kin_energie))
print('kinetic energy free symbols', kin_energie.free_symbols)
print('potential energy DS: ', me.find_dynamicsymbols(pot_energie))
print('potential energy free symbols', pot_energie.free_symbols)

**Kane's equations**

- determine the external forces, here only gravitational forces
- get the equation for the reaction forces. They (of course) depend on the accelerations of the masses, hence on $rhs = MM^{-1} \cdot force$. It is calculated numerically later on.
- add the term to calculate the  X - position of CP at the bottom of force. Recall we got a differential equation for x(t).
- enlarge the mass matrix appropriately
- convert the sympy functions to numpy functions, using *lambdify(..)*

In [ ]:
FL = [(Dmc, -m*g*N.y), (Dmco, -mo*g*N.y), (Dmc, fx*N.x + fy*N.y), (A2, -reibung*u1*A2.z)]
kd = [u1 - q1.diff(t)]
q = [q1]
u = [u1]
aux = [auxx, auxy]

KM = me.KanesMethod(N, q_ind=q, u_ind=u, kd_eqs=kd, u_auxiliary=aux)
(fr, frstar) = KM.kanes_equations(BODY, FL)
MM = KM.mass_matrix_full
force = KM.forcing_full

# Reaction forces
eingepraegt = KM.auxiliary_eqs.subs({sm.Derivative(u1, t): rhs1, sm.Derivative(x, t): rhs3})
print('eingepraegt DS', me.find_dynamicsymbols(eingepraegt))
print('eingepraegt free symbols', eingepraegt.free_symbols)
print('eingepraegt has {} operations'.format(sum([eingepraegt[i].count_ops(visual=False) for i in range(len(eingepraegt))])), '\n')

# Add rhs3 at the bottom of force, to get d/dt(x) = rhs3. This is to numerically integrate x(t)
force = sm.Matrix.vstack(force, sm.Matrix([rhs3])).subs({sm.Derivative(x, t): rhs3, fx: 0., fy: 0.}) 
print('force DS', me.find_dynamicsymbols(force))
print('force free symbols', force.free_symbols)
print('force has {} operations'.format(sum([force[i].count_ops(visual=False) for i in range(len(force))])), '\n')

# Enlarge MM properly
MM = sm.Matrix.hstack(MM, sm.Matrix([0., 0.])).subs({sm.Derivative(x, t): rhs3})
MM = sm.Matrix.vstack(MM, sm.Matrix([0., 0., 1.]).T)
print('MM DS', me.find_dynamicsymbols(MM))
print('MM free symbols', MM.free_symbols)
print('MM has {} operations'.format(sum([MM[i, j].count_ops(visual=False) for i in range(MM.shape[0]) for j in range(MM.shape[1])])), '\n')

# Lambdification. Turning symbolic expressions into numpy functions.
pL = [m, mo, g, r, iZZ, alpha, beta, amplitude, frequenz, reibung]
qL = q + u + [x]
F = [fx, fy]

MM_lam = sm.lambdify(qL + pL, MM, cse=True)
force_lam = sm.lambdify(qL + pL, force, cse=True)

CP_pos_lam = sm.lambdify(qL + pL, CP_pos, cse=True)
Dmc_pos_lam = sm.lambdify(qL + pL, Dmc_pos, cse=True)
Dmco_pos_lam = sm.lambdify(qL + pL, Dmco_pos, cse=True)

# will be solved for F numerically later.
eingepraegt_lam = sm.lambdify(F + qL + pL + [rhs1], eingepraegt, cse=False) 

#this is needed to plot the shape of the street
strasse_lam = sm.lambdify([x] + pL,  gesamt, cse=True)

kin_lam = sm.lambdify(qL + pL, kin_energie, cse=True)
# kin1_lam is needed further down for fsolve, where I have to solve for u1
kin1_lam = sm.lambdify([u1] + [q1, x] + pL, kin_energie, cse=True)
pot_lam = sm.lambdify(qL + pL, pot_energie, cse=True)

r_max_lam = sm.lambdify([x] + pL, r_max, cse=True)

print('it took {:.3f} sec to establish Kanes equations'.format(time.time() - start))

**Numerical integration**

A.\
Parameter / initial values
- $q_{11}, u_{11}$: rotation of the disc and its speed
- $x_1, u_{x1}$: loction of the contact point and the spped of successive contact points

- $m_1$: mass of the disc
- $m_{o1}$: mass of the observer
- $i_{ZZ1}$: moment of inertia of the disc aroud the Z axis
- $\alpha_1, \beta_1$: location of the observer relative to the center of the disc
- $amplitude_1, frequenz_1$: parameters of the street
- $reibung_1$: friction

While it makes sense to call these values similar to their names when setting up Kane's equations, **avoid** the **same** name: The symbols / dynamic symbols get overwritten, with unintended consequences.\
In order to find a second contaxct point, at least the way I could come up with, *step_size* and *max_step* have to be small, slowing down the integration.

In [ ]:
# Input parameters 
#==========================================================
m1 = 1.
mo1 = 1.
r1 =  2.
alpha1, beta1 = 0., 0.99
amplitude1 = 1.
frequenz1 = .9   # the smaller this number, the more 'even' the street   
reibung1 = 0.        # Friction
intervall = 10.     # time inverval of integration is [0., intervall]

q11 = 0.             # starting angle. As the disc is symmetric about this angle, it plays no real role
u11 = 3.5             # starting angular velocity of disc.
x1  = 7.5             # Starting X position of disc. 

step_size = 0.01    # Stepsize to be used to search for the second CP
max_step  = 0.01     # Max. stepsize for solve_ivp 

punkte    = 500      # Determines the number of times given out by the solution of solve_ivp
#==========================================================
schritte = int(intervall * punkte)

iZZ1 = 1/2 * m1 * r1**2
pL_vals = [m1, mo1,  9.8, r1, iZZ1, alpha1, beta1, amplitude1, frequenz1, reibung1]

y0 = [q11, u11, x1]
print('Arguments')
print('[m, mo1,  g, r, iZZ, iXY, alpha, beta, amplitude, frequenz, reibung]')
print(pL_vals, '\n')
print('[q11, u11, x1]')
print(y0, '\n')

startwert = y0[2]   # just needed for the plots below
startomega = y0[1]  #  dto.

#find the largest admissible r, given strasse, amplitude, frequenz
def func(x, args):
# just needed to get the arguments matching for minimize
    return np.abs(r_max_lam(x, *args))

x0 = 0.1            # initial guess
minimal = minimize(func, x0, pL_vals)

if pL_vals[3] < (x111 := minimal.get('fun')):
    print('selected radius = {} is less than maximally admissible radius = {:.2f}, hence o.k.'
          .format(pL_vals[3], x111), '\n')
else:
    print('selected radius {} is larger than admissible radius {:.2f}, hence a second contact point may happen.'
          .format(pL_vals[3], x111), '\n')

# At the initial position, I do not want a second contact point nearby.
for vorzeichen in (-1., 1.):   # determines, which direction we must look at for CP2
    bereich    = np.linspace(x1 - vorzeichen*step_size, x1 - vorzeichen * 2. * (r1+1.), int(2.*r1/step_size))
    
    for xh1 in bereich:
        if abstand2_lam(x1, xh1, r1, amplitude1, frequenz1) <= r1:
            raise Exception('change starting point')
        else:
            pass      

When a new contact point is found, the moment of inertia w.r.t. this new contact point is in general different from the moment of inertia w.r.t. the old contact point. As the total energy must remain constant, $\frac{d}{dt}q(t)$ must in general change discontinuously.\
This function is used to calculate the new angular velocity $\frac{d}{dt}q(t)$.

In [ ]:
def funcCP(x0, args):
    return kin1_lam(x0, args[0], args[1], *args[5]) - kin_lam(args[2], args[3], args[4], *args[5])

B.\
Actual **numerical integration** starts here.

The function *event* is needed to stop *solve_ivp* when a second contact point is found, see the documentation of *solve_ivp* for details.\
Of course, I only have to look for second contact points in the direction of the movement at the time.\
Once a second contact point is found, the numerical integration is stopped, and its results stored in *ergebnis*. Then a new numrical integration is started with new initial conditions.\
I need $CP_{2x}$ for the next integration, but I cannot return it, as *solve_ivp* needs a defined output of the *event* function. Hence I make it a global variable.

NOTES:\
The *events* keyword of *solve_ivp* requires a continuous function $f(t, y, args)$ with {event happened at $t_0, y_0$} <=> $f(t_0, y_0, args) = 0.$\
I my case, with a numerical criterium, the right way to do it is like this, **a helpful person in *Stack Overflow* explained it to me**:\
As long as the event has not occured, the function returns $-1$. When the event did occur, it returns $+1$. This seems to work fine.

In [ ]:
start1 = time.time()
schritte = int(intervall * punkte)
times = np.linspace(0, intervall, schritte)
CP2x = 0.
y0 = [q11, u11, x1]
zaehl_event = 0
#==============================================
def event(t, y, args):
    global CP2x, zaehl_event
    zaehl_event += 1
    vorzeichen = np.sign(y[1])  # determines, which direction we must look at for CP2
    bereich    = np.linspace(y[2] - 5. * vorzeichen*step_size, y[2] - vorzeichen * 3. * r1, 
                    int(2.*r1/step_size))
    
    for xh1 in bereich:
        try:
            if abstand2_lam(y[2], xh1, r1, amplitude1, frequenz1) <= r1:
                raise Rausspringen()
            else:
                pass
                
        except:
# event_info = True shows, how solve_ivp searches for the exact location where the even took place
            if event_info == True:
                print(f'second CP, as {(abstand2_lam(y[2], xh1, r1, amplitude1, frequenz1) - r1):.3f} <=0, at time {t:.3f} and location {xh1:.3f}')
            CP2x = xh1
            return 1.   # 0 a second contact point was found
        
    return -1. # 1 no second contact point

#===============================================
event.terminal = True   # if True, this stops the integration if event occurs
event_info     = False  # if True, data related to the occurence of events are printed
#===============================================


def gradient(t, y, args):
    vals = np.concatenate((y, args))
    sol = np.linalg.solve(MM_lam(*vals), force_lam(*vals))
    return np.array(sol).T[0]

runtime    = 0.
starttime  = 0.
starttime1 = 0.
ergebnis   = []
event_dict = {-1: 'Integration failed', 0: 'Integration finished successfully', 
            1: 'some termination event'}

# here the 'piecewise' integration starts.
while starttime < intervall:
    resultat1 = solve_ivp(gradient, (starttime, float(intervall)), y0, t_eval=times, args=(pL_vals,), 
            atol=1.e-7, rtol=1.e-7, max_step=max_step, events=event, method='Radau')

    resultat = resultat1.y.T
    if event_info == True:
        print(event_dict[resultat1.status], ' im loop message is:', resultat1.message)
        print('resultat shape', resultat.shape)
    
    if resultat1.y_events[0].size > 0.:
        height = strasse_lam(resultat1.y_events[0][0][2], *pL_vals )
        if event_info  == True:
            print(f'generalized coordinates at exit time, height of CP        {resultat1.y_events[0][0][0]:.3f} ' +
                 f'{resultat1.y_events[0][0][1]:.3f}  {resultat1.y_events[0][0][2]:.3f}  {height:.3f}')
            
        hilfs0 = resultat1.y_events[0][0][0]
        hilfs1 = resultat1.y_events[0][0][1]
        hilfs2 = resultat1.y_events[0][0][2]
        args1  = [hilfs0, CP2x] + [hilfs0, hilfs1, hilfs2, pL_vals]
        x0     = hilfs1

# Force the total energy to be constant.        
# I iterate here, this sometimes improves the accuracy, as can be seen if you print the x0.
        for _ in range(3):
            hilfs3 = fsolve(funcCP, x0, args1)
            hilfs3 = hilfs3[0]
            x0     = hilfs3

        y0 = [resultat1.y_events[0][0][0], hilfs3, CP2x]
        height = strasse_lam(CP2x, *pL_vals)
        if event_info == True:
            print(f'initial values for the next integration, height of new CP {hilfs0:.3f} {hilfs3:.3f}' + 
                  f' {CP2x:.3F}  {height:.3f}' + '\n')
        starttime = resultat1.t_events[0][0]
        
        schritte = int((intervall - starttime) * punkte)
        times    = np.linspace(starttime, intervall, schritte)
    
        ergebnis.append(resultat)
    else:
        break

print("To numerically integrate an intervall of {} sec took {:.5f} sec "
      .format(intervall, time.time() - start1))
print('message is', resultat1.message)
# stack the individual results of the various integrations, to get the complete results.
resultat = np.vstack(ergebnis)
print(resultat.shape)

# set these values for the subsequent plots below.
schritte = resultat.shape[0]
times = np.linspace(0., intervall, schritte)
print('how often did solve_ivp call event:', zaehl_event)

Plot the generalized coordinates and speeds

In [ ]:
Dmc_X = np.empty(schritte)
Dmc_Y =np.empty(schritte)
for i in range(schritte):
    Dmc_X[i], Dmc_Y[i] = Dmc_pos_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals)

fig, ax = plt.subplots(figsize=(10, 5))
for i, j in zip(range((resultat.shape[1])), ('rotational angle', 'rotational speed', 'displacement')):
    ax.plot(times, resultat[:, i], label=j)
ax.set_title('Coordinates, friction = {}'.format(reibung1))
ax.set_xlabel('time (sec)')
ax.legend();

Plot the reaction forces 

In [ ]:
# RHS calculated numerically, too large to do it symbolically. Needed for reaction forces.
RHS1 = np.zeros((schritte, resultat.shape[1]))
for i in range(schritte):
    RHS1[i, :] = np.linalg.solve(MM_lam(*[resultat[i, j]for j in range(resultat.shape[1])], *pL_vals), 
        force_lam(*[resultat[i, j] for j in range(resultat.shape[1])], 
        *pL_vals)).reshape(resultat.shape[1])
print('RHS1 shape', RHS1.shape)

def func (x11, *args):
# just serves to make the arguments compatible between fsolve and eingepraegt_lam
    return eingepraegt_lam(*x11, *args).reshape(len(F))

kraft = np.empty((schritte, len(F)))
x0 = tuple([1. for i in range(len(F))])   # initial guess
for i in range(schritte):
    for _ in range(2):
        y00 = [resultat[i, j] for j in range(resultat.shape[1])]
        args = tuple((y00 + pL_vals + [RHS1[i, 1]]))
        A = fsolve(func, x0, args=args).reshape(len(F)) # numerically find fx, fy
        x0 = tuple(A)      # updated initial guess, should improve convergence
    kraft[i] = A        
        
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(times, kraft[:, 0], label = 'Fx')
ax.plot(times, kraft[:, 1], label = 'Fy')
ax.set_title('Reaction forces on contact point, friction = {}'.format(reibung1))
ax.set_xlabel('time (sec)')
ax.legend();

Plot the **energies** of the system.

In [ ]:
kin_np = np.empty(schritte)
pot_np = np.empty(schritte)
total_np = np.empty(schritte)

for i in range(schritte):
    kin_np[i] = kin_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals)
    pot_np[i] = pot_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals)
    total_np[i] = kin_np[i] + pot_np[i]
if pL_vals[-1] == 0.:
    print('Max deviation from constant of total energy is {:.2e} % of max total energy'
          .format((max(total_np) - min(total_np))/max(total_np) * 100.))
    print('Max absolute deviation from constant of total energy is {:.2e} Nm'
          .format((np.max(total_np) - np.min(total_np))))
fig, ax = plt. subplots(figsize=(10, 5))
ax.plot(times, kin_np, label='kinetic energy')
ax.plot(times, pot_np, label='pos energy')
ax.plot(times, total_np, label='total energy')
ax.set_title('Energy of the disc, friction = {} frequenz = {}'.format(reibung1,frequenz1))
ax.set_xlabel('time (sec)')
ax.legend();

Plot the street and the extreme positions of the disc

In [ ]:
# plot the street, and the extremes, of the position of the disc.
fig, ax = plt.subplots(figsize=(10, 5))
links = np.min(resultat[:, 2])
rechts = np.max(resultat[:, 2])
ruhe = np.mean([resultat[-30::, 2]])    # get approx. rest position of wheel
maximal = max(np.abs(links), np.max(rechts))
times1 = np.linspace(-maximal-5, maximal+5, schritte)
ax.plot(times1, strasse_lam(times1, *pL_vals)  , label='Strasse')
if pL_vals[-1] != 0.:
    ax.axvline(ruhe,ls = '--', color='red', label='approx. fimal pos. of wheel')
ax.axvline(links,ls = '--', color='green', label='leftmost pos. of wheel')
ax.axvline(rechts,ls = '--', color='black', label='rightmost pos. of wheel');
ax.axvline(startwert, ls='--', color='orange', label='starting position of wheel')
if startomega > 0.:
    richtung = 'left'
else:
    richtung = 'right'
text = 'Wheel has speed ' + str(np.abs(startomega)) + ' units to the ' + richtung
plt.title(text + ', friction = {} '.format(reibung1))
plt.xlabel('horizontal distance (m)')
plt.ylabel('elevation (m)')
ax.legend();

**Animation**

The blue dot represents the observer.
HTML(...) is needed for the animation to show on an iPad, no idea, whether needed on other machines. It is SLOW!\
The number of points considered are *zeitpunkte*. If it builds too slowly, this number may be reduced - possibly at the loss of some accuracy.

In [ ]:
# reduce the number of points of time to around zeitpunkte
times2 = []
resultat2 = []
index2 = []

#=======================
zeitpunkte = 400
#=======================

reduction = max(1, int(len(times)/zeitpunkte))

for i in range(len(times)):
    if i % reduction == 0:
        times2.append(times[i])
        resultat2. append(resultat[i])
 
schritte2 = len(times2)
print(f'animation used {schritte2} points in time')
resultat2 = np.array(resultat2)
times2 = np.array(times2)

# Location of the center of the disc
Dmcx = np.empty(schritte2)
Dmcy =np.empty(schritte2)
Dmcox = np.empty(schritte2)
Dmcoy =np.empty(schritte2)

for i in range(schritte2):
    Dmcx[i], Dmcy[i] = Dmc_pos_lam(*[resultat2[i, j] for j in range(resultat2.shape[1])], *pL_vals)
    Dmcox[i], Dmcoy[i] = Dmco_pos_lam(*[resultat2[i, j] for j in range(resultat2.shape[1])], *pL_vals)

# needed to give the picture the right size.
xmin = min([resultat2[i, 2] for i in range(schritte2)])
xmax = max([resultat2[i, 2] for i in range(schritte2)])

ymin = min([strasse_lam(resultat2[i, 2], *pL_vals) for i in range(schritte2)]) 
ymax = max([strasse_lam(resultat2[i, 2], *pL_vals) for i in range(schritte2)]) 

# Data to draw the uneven street
cc = r1
strassex = np.linspace(xmin - 3*cc, xmax + 3.*cc, schritte2)
strassey = [strasse_lam(strassex[i], *pL_vals) for i in range(len(strassex))]

def animate_pendulum(times, x1, y1, x2, y2):    
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'aspect': 'equal'})
    
    ax.axis('on')
    ax.set_xlim(xmin - 3.*cc, xmax + 3.*cc)
    ax.set_ylim(ymin - 3.*cc, ymax + 3.*cc)
    ax.plot(strassex, strassey)
    ax.set_xlabel('horizontal distance (m)')
    ax.set_ylabel('elevation (m)')
    
    line1, = ax.plot([], [], 'o-', lw=0.5)
    line2  = ax.axvline(resultat2[0, 2], linestyle='--')                         # vertical tracking line
    line3  = ax.axhline(strasse_lam(resultat2[0, 2], *pL_vals), linestyle = '--') # horizontal trackimg line
    line4, = ax.plot([], [], 'bo', markersize=5) # the dot on the disc, to show it is rotating
    
    elli = patches.Circle((x1[0], y1[0]), radius = r1, fill=True, color='red', ec='black')
    ax.add_patch(elli)

    def animate(i):        
        ax.set_title('running time {:.2f} sec, friction ={}'.format(times2[i], reibung1), fontsize=15)
        
        elli.set_center((x1[i], y1[i]))
        elli.set_height(2.*r1)
        elli.set_width(2.*r1)
        elli.set_angle(np.rad2deg(resultat[i, 0]))
                       
        line1.set_data([x1[i]], [y1[i]])                  # center of the disc
        line2.set_xdata([resultat2[i, 2]])  # dashed line to mark the contact point
        line3.set_ydata([strasse_lam(resultat2[i, 2], *pL_vals)])    #            dto. 
        line4.set_data([x2[i]], [y2[i]])
        return line1, line2, line3, line4, 

    anim = animation.FuncAnimation(fig, animate, frames=len(times),
                                   interval=1000*max(times) / len(times),
                                   blit=True)
    plt.close(fig)
    return anim

anim = animate_pendulum(times2, Dmcx, Dmcy, Dmcox, Dmcoy)
print('it took {:.3f} sec to run the program, before HTML(..)'.format(time.time() - start))
HTML(anim.to_jshtml())    # needed, when run on an iPad, I know no other way to do it. It is SLOW!